<a href="https://colab.research.google.com/github/SunkaraboinaPraveenKumar/Machine_Learning_Projects/blob/main/Weather_Research_Agent_Llama_Index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata
import os
os.environ['GROQ_API_KEY']=userdata.get('groq_api_key')
os.environ['WEATHER_API_KEY']=userdata.get('WEATHER_API_KEY')

In [2]:
WEATHER_API_KEY=os.getenv('WEATHER_API_KEY')

In [3]:
!pip install requests llama-index llama-index-llms-groq -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.9/809.9 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.1/253.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.4 MB/s eta 0:00:00


In [4]:
import requests
from llama_index.core.agent import ReActAgent
from llama_index.llms.groq import Groq
from llama_index.core.tools import FunctionTool

In [5]:
llm=Groq(
    model="llama-3.3-70b-versatile",
    temperature=0
)

In [7]:
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = WEATHER_API_KEY

In [10]:
city = 'Hyderabad'
units = 'metrics'
params={
    'q':city,
    'appid':API_KEY,
    'units':units
}
response = requests.get(url=BASE_URL,params=params)

In [11]:
print(response.json())

{'coord': {'lon': 78.4744, 'lat': 17.3753}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 310.38, 'feels_like': 310.7, 'temp_min': 309.88, 'temp_max': 310.38, 'pressure': 1004, 'humidity': 28, 'sea_level': 1004, 'grnd_level': 943}, 'visibility': 6000, 'wind': {'speed': 3.09, 'deg': 140}, 'clouds': {'all': 40}, 'dt': 1745410831, 'sys': {'type': 1, 'id': 9214, 'country': 'IN', 'sunrise': 1745367885, 'sunset': 1745413427}, 'timezone': 19800, 'id': 1269843, 'name': 'Hyderabad', 'cod': 200}


Weather Forecast Function

In [12]:
import requests
BASE_URL = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = WEATHER_API_KEY
units = 'metrics'

def get_weather_forecast(city:str)->dict:
    """This function returns the current weather forecast for the given city."""
    params={
        'q':city,
        'appid':API_KEY,
        'units':units
    }
    response = requests.get(url=BASE_URL,params=params)
    return response.json()



In [13]:
output = get_weather_forecast('Hyderabad')

In [14]:
# function tool
weather_tool = FunctionTool.from_defaults(fn=get_weather_forecast)

In [16]:
tools_list=[weather_tool]

In [18]:
agent = ReActAgent.from_tools(
    tools=tools_list,
    llm=llm,
    verbose=True
)

In [20]:
response = agent.chat("What is the temperature in Hyderabad?")

> Running step b25a313f-33a3-4a92-ad6c-d28284f7beb8. Step input: What is the temperature in Hyderabad?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_weather_forecast
Action Input: {'city': 'Hyderabad'}
Observation: {'coord': {'lon': 78.4744, 'lat': 17.3753}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'base': 'stations', 'main': {'temp': 310.38, 'feels_like': 310.7, 'temp_min': 309.88, 'temp_max': 310.38, 'pressure': 1004, 'humidity': 28, 'sea_level': 1004, 'grnd_level': 943}, 'visibility': 6000, 'wind': {'speed': 3.09, 'deg': 140}, 'clouds': {'all': 40}, 'dt': 1745411147, 'sys': {'type': 1, 'id': 9214, 'country': 'IN', 'sunrise': 1745367885, 'sunset': 1745413427}, 'timezone': 19800, 'id': 1269843, 'name': 'Hyderabad', 'cod': 200}
> Running step 0c392f70-eedf-45cd-a854-feb359ff8197. Step input: None
Observation: Error: Could not parse output. Please follow the tho

In [21]:
print(response.response)

The temperature in Hyderabad is 37.23 degrees Celsius.
